In [30]:
import numpy as np
import csv
import pandas as pd
import pyprind
import scipy.sparse as spr
import graphlab as gl
import gzip
from sklearn.cluster import KMeans
from math import isnan
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error
sns.set_style("whitegrid")
sns.set_context("poster")
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
%matplotlib inline

## Load Data

In [31]:
train_file = 'data/train.csv'
test_file  = 'data/test.csv'
profiles_file = 'data/profiles.csv'
artists_file = 'data/artists.csv'
artists_file_info = 'data/artist_infos.json'

In [32]:
df = pd.read_csv(train_file, header=0)

## Baseline Model

In [33]:
## Train-Test Split
trainlist=[]
validatelist=[]
take=4
i = 0
for k, v in df.groupby('user'):
    if v.artist.count() > 10:
        validatelist.append(v[-take:]) 
        trainlist.append(v[:-take])
    else:
        trainlist.append(v)
traindf=pd.concat(trainlist)
validatedf=pd.concat(validatelist)
print traindf.shape, validatedf.shape

(3254976, 3) (899828, 3)


In [34]:
pd.DataFrame.to_csv(traindf, 'data/traindf')
pd.DataFrame.to_csv(validatedf, 'data/validatedf')

In [35]:
traindf = pd.read_csv("data/traindf")
validatedf = pd.read_csv("data/validatedf")

In [36]:
ybar = traindf.plays.mean()

uuids=traindf.user.unique()#unique-users
uiids=traindf.artist.unique()#unique-items
uuidmap={v:k for k,v in enumerate(uuids)}#of length U
uiidmap={v:k for k,v in enumerate(uiids)}#of length M

groupby_user = traindf.groupby('user')
user_biases = dict()
user_means = dict()
user_medians = dict()
for id in uuids:
    user_mean = groupby_user.get_group(id).plays.mean()
    user_biases[id] = user_mean - ybar
    user_means[id] = user_mean
    user_medians[id] = groupby_user.get_group(id).plays.median()

groupby_item = traindf.groupby('artist')
item_biases = dict()
item_means = dict()
for id in uiids:
    item_mean = groupby_item.get_group(id).plays.mean()
    item_biases[id] = item_mean - ybar
    item_means[id]  = item_mean
    

train_avgs={'mean':ybar, 'users':user_biases, 'items':item_biases}

predictions={}
user_biases_validate = validatedf['user'].apply(lambda x: user_biases[x])
item_biases_validate = validatedf['artist'].apply(lambda x: item_biases[x])

predictions['baseline'] = (ybar + user_biases_validate + item_biases_validate).values
predictions['user_mean'] = validatedf['user'].apply(lambda x: user_means[x])
predictions['user_median'] = validatedf['user'].apply(lambda x: user_medians[x])



## GraphLab Create

In [37]:
#### Load Data as SFrames ####

In [38]:
sf_train = gl.SFrame.read_csv("data/traindf", header=True, verbose=False, column_type_hints={"plays":int})
sf_train.remove_column('X1')
sf_train.rename({'user':'user_id', "artist": "item_id"})

user_id,item_id,plays
00000c289a1829a808ac09c00daf10bc3c4e223b ...,c995a379-60b9-404b-bd97-a7e2de0751d3 ...,198
00000c289a1829a808ac09c00daf10bc3c4e223b ...,b071f9fa-14b0-4217-8e97-eb41da73f598 ...,150
00000c289a1829a808ac09c00daf10bc3c4e223b ...,7b885d42-3c41-4f43-9944-a5855ec5155e ...,361
00000c289a1829a808ac09c00daf10bc3c4e223b ...,9fdaa16b-a6c4-4831-b87c-bc9ca8ce7eaa ...,168
00000c289a1829a808ac09c00daf10bc3c4e223b ...,fb01635c-51fc-4cad-b71f-62e18bb3433b ...,231
00000c289a1829a808ac09c00daf10bc3c4e223b ...,83998f9c-846b-4294-aede-d7735531c901 ...,232
00000c289a1829a808ac09c00daf10bc3c4e223b ...,e8374874-4178-4869-b92e-fef6bf30dc04 ...,302
00000c289a1829a808ac09c00daf10bc3c4e223b ...,144ef525-85e9-40c3-8335-02c32d0861f3 ...,182
00000c289a1829a808ac09c00daf10bc3c4e223b ...,3d6bbeb7-f90e-4d10-b440-e153c0d10b53 ...,717
00000c289a1829a808ac09c00daf10bc3c4e223b ...,8000598a-5edb-401c-8e6d-36b167feaf38 ...,222


In [39]:
sf_validate = gl.SFrame.read_csv("data/validatedf", header=True, verbose=False, column_type_hints={"plays":int})
sf_validate.remove_column('X1')
sf_validate.rename({'user':'user_id', "artist": "item_id"})

user_id,item_id,plays
00001411dc427966b17297bf4d69e7e193135d89 ...,e105c272-b5d7-4135-82ef-d60bded54345 ...,1244
00001411dc427966b17297bf4d69e7e193135d89 ...,9fa07bb4-8312-4cd7-a19a-4fdbc5797148 ...,2641
00001411dc427966b17297bf4d69e7e193135d89 ...,ffb18e19-64a4-4a65-b4ce-979e00c3c69d ...,622
00001411dc427966b17297bf4d69e7e193135d89 ...,451f9db1-f75f-44f9-b218-f8bdf22035a1 ...,2427
000063d3fe1cf2ba248b9e3c3f0334845a27a6bf ...,50eec634-7c42-41ee-9b1f-b41d9ca28b26 ...,8
000063d3fe1cf2ba248b9e3c3f0334845a27a6bf ...,2449300a-6ca7-45da-8102-22789d256475 ...,14
000063d3fe1cf2ba248b9e3c3f0334845a27a6bf ...,61ed9c9c-79eb-4e8f-8015-bd599ac0ab49 ...,11
000063d3fe1cf2ba248b9e3c3f0334845a27a6bf ...,66fc5bf8-daa4-4241-b378-9bc9077939d2 ...,4
00007a47085b9aab8af55f52ec8846ac479ac4fe ...,69158f97-4c07-4c4e-baf8-4e4ab1ed666e ...,407
00007a47085b9aab8af55f52ec8846ac479ac4fe ...,10adbe5e-a2c0-4bf3-8249-2b4cbf6e6ca8 ...,149


In [40]:
sf_test = gl.SFrame.read_csv(test_file, header=True, verbose=False)
sf_test.rename({'user':'user_id', "artist": "item_id"})

Id,user_id,item_id
1,306e19cce2522fa2d39ff5dfc870992100ec22d2 ...,4ac4e32b-bd18-402e-adad-ae00e72f8d85 ...
2,9450d351278df4938bdea4ed86aec940a4e927ac ...,1f574ab1-a46d-4586-9331-f0ded23e0411 ...
3,801909d6955f59033c88595d3d7f8a6a5dcd53cc ...,3eb72791-6322-466b-87d3-24d74901eb2d ...
4,e3ed47445c127fbeff47fb58f6bbf2f3b4535d82 ...,61604b45-8a91-4e33-a1b6-45d7b1fec4e5 ...
5,a73f46652103f3a5f7429159310f6928f79644aa ...,5dfdca28-9ddc-4853-933c-8bc97d87beec ...
6,55f1c89e3a102de38a0bdfcb1fe660b028c5c0af ...,ef58d4c9-0d40-42ba-bfab-9186c1483edd ...
7,7ad7619f38bf52de421b795f3159949422b5bbee ...,a3cb23fc-acd3-4ce0-8f36-1e5aa6a18432 ...
8,2a1ab291185cfc6fb996556481e178666e9731f9 ...,5dfdca28-9ddc-4853-933c-8bc97d87beec ...
9,3f407fff902ab403f06668f6be3d10bb9e9a02cd ...,000fc734-b7e1-4a01-92d1-f544261b43f5 ...
10,2eea5a2b68e6e9e0d9dd6b3c1c0e1e5840fc9ce3 ...,84783313-2a3f-4f17-ab9a-425a5d527879 ...


In [41]:
#### Models ####

In [42]:
#### Item Similarity Models (Jaccard similarity) ####
item_sim_model_jaccard = gl.item_similarity_recommender.create(sf_train, user_id="user_id", item_id="item_id", target="plays")

result_item_sim_model_jaccard = item_sim_model_jaccard.predict(sf_validate)

predictions['item_sim_model_jaccard'] = result_item_sim_model_jaccard

Recsys training: model = item_similarity

Preparing data set.

Data has 3254976 observations with 233286 users and 2000 items.

Data prepared in: 5.65549s

Computing item similarity statistics:

...

| 2000            | 0.980958        |

+-----------------+-----------------+

Finished training in 1.44074s

Finished prediction in 3.06442s

Finished prediction in 0.862076s

In [43]:
#### Item Similarity Models (Cosine similarity) ####
item_sim_model_cosine = gl.item_similarity_recommender.create(sf_train, user_id="user_id", item_id="item_id", target="plays", similarity_type="cosine")

result_item_sim_model_cosine = item_sim_model_cosine.predict(sf_validate)

predictions['item_sim_model_cosine'] = result_item_sim_model_cosine

Recsys training: model = item_similarity

Preparing data set.

Data has 3254976 observations with 233286 users and 2000 items.

Data prepared in: 6.2959s

Computing item similarity statistics:

...

| 2000            | 0.500115        |

+-----------------+-----------------+

Finished training in 0.904401s

Finished prediction in 2.37114s

Finished prediction in 0.975346s

In [44]:
#### Item Similarity Models (Pearson similarity) ####
item_sim_model_pearson = gl.item_similarity_recommender.create(sf_train, user_id="user_id", item_id="item_id", target="plays", similarity_type="pearson")

result_item_sim_model_pearson = item_sim_model_pearson.predict(sf_validate)

predictions['item_sim_model_pearson'] = result_item_sim_model_pearson

Recsys training: model = item_similarity

Preparing data set.

Data has 3254976 observations with 233286 users and 2000 items.

Data prepared in: 6.59028s

Computing item similarity statistics:

...

| 2000            | 0.837459        |

+-----------------+-----------------+

Finished training in 1.36061s

Finished prediction in 2.00195s

Finished prediction in 0.870869s

In [45]:
#### Popularity Model  ####
popularity_model = gl.popularity_recommender.create(sf_train, user_id="user_id", item_id="item_id", target="plays")

result_popularity_model = popularity_model.predict(sf_validate)

predictions['popularity_model'] = result_popularity_model

Recsys training: model = popularity

Preparing data set.

Data has 3254976 observations with 233286 users and 2000 items.

Data prepared in: 6.32245s

3254976 observations to process; with 2000 unique items.

In [47]:
#### Factorization Recommender ####
user_info = gl.SFrame.read_csv(profiles_file, header=True, verbose=False).dropna()
user_info.rename({"user": "user_id"})

item_info = gl.SFrame.read_csv(artists_file, header=True, verbose=False).dropna()
item_info.rename({"artist": "item_id"})

factorization_model = gl.factorization_recommender.create(sf_train, target='plays', \
        user_data=user_info, \
        item_data=item_info, max_iterations=50, num_factors=5, regularization=0.0001)

predictions['factorization_model'] = factorization_model.predict(sf_validate)

Recsys training: model = factorization_recommender

Preparing data set.

Data has 3254976 observations with 233286 users and 2000 items.

Data prepared in: 5.38684s

Training factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 506582

Final training RMSE: 711.707

In [48]:
#### Alternating Least Squares Model ####
als_model = gl.factorization_recommender.create(sf_train, target='plays', \
        solver='als')

predictions['als_model'] = als_model.predict(sf_validate)

Recsys training: model = factorization_recommender

Preparing data set.

Data has 3254976 observations with 233286 users and 2000 items.

Data prepared in: 5.70589s

Training factorization_recommender for recommendations.

...

| 48      | 1m 36s       | 174.937                |

| 49      | 1m 38s       | 174.681                |

| FINAL   | 1m 38s       | 174.681                |

+---------+--------------+------------------------+

Optimization Complete: Iteration limit reached.

## Comparing Models:

In [ ]:
validatedf.plays

In [49]:
print "Mean Absolute Errors:"
print " "
print "Baseline: \t\t\t %s" % mean_absolute_error(validatedf.plays, predictions['baseline'])
print "User Mean: \t\t\t %s" % mean_absolute_error(validatedf.plays, predictions['user_mean'])
print "User Median: \t\t\t %s" % mean_absolute_error(validatedf.plays, predictions['user_median'])
print "Item Similarity (Jaccard): \t %s" % mean_absolute_error(validatedf.plays, pd.Series(predictions['item_sim_model_jaccard']))
print "Item Similarity (Cosine): \t %s" % mean_absolute_error(validatedf.plays, pd.Series(predictions['item_sim_model_cosine']))
print "Item Similarity (Pearson): \t %s" % mean_absolute_error(validatedf.plays, pd.Series(predictions['item_sim_model_pearson']))
print "Popularity Model: \t\t %s" % mean_absolute_error(validatedf.plays, pd.Series(predictions['popularity_model']))
print "Factorization Model: \t\t %s" % mean_absolute_error(validatedf.plays, pd.Series(predictions['factorization_model']))
print "Alternating Least Squares Model: %s" % mean_absolute_error(validatedf.plays, pd.Series(predictions['als_model']))


Mean Absolute Errors:
 
Baseline: 			 180.750413294
User Mean: 			 162.44725936
User Median: 			 137.265611872
Item Similarity (Jaccard): 	 190.759273481
Item Similarity (Cosine): 	 194.184909668
Item Similarity (Pearson): 	 201.058108789
Popularity Model: 		 233.484260639
Factorization Model: 		 226.01765518
Alternating Least Squares Model: 633.626737926


## Code to store predictions

In [ ]:
"""with open("data/item_sim_model_jaccard.csv", 'w') as soln_fh:
    soln_csv = csv.writer(soln_fh,
                          delimiter=',',
                          quotechar='"',
                          quoting=csv.QUOTE_MINIMAL)
    soln_csv.writerow(['Id', 'plays'])
    loop_size = 4154804
    mybar = pyprind.ProgBar(loop_size)
    for i in range(4154804):
        soln_csv.writerow([i + 1, item_sim_model_jaccard[i]])
        mybar.update()"""

# MISC
### Old Ridge Regression Code

In [ ]:
"""# Design Matrix
from sklearn.linear_model import Ridge
features=np.concatenate([uuids,uiids])
features.shape

def getmats(indf):
    stvals=indf[['user', 'artist', 'plays']].values
    designm=np.zeros((stvals.shape[0], features.shape[0]))
    ratings=np.zeros(stvals.shape[0])
    loop_size = len(stvals)
    mybar = pyprind.ProgBar(loop_size)
    for i, row in enumerate(stvals):
        designm[i,:]=np.concatenate([1*(row[0]==uuids), 1*(row[1]==uiids)])
        ratings[i]=row[2]
        mybar.update()
    return designm, ratings



#Carrying out the ridge regression
designm, ratings = getmats(train_df)

validatedm, validaterats = getmats(validate_df)



#alphas=[0.01, 0.1, 10, 100, 1000]
alphas=[7,8,9,10,11,12,13]
vdict={}
rdict={}
loop_size = len(alphas)
mybar = pyprind.ProgBar(loop_size)
for a in alphas:
    regr=Ridge(alpha=a).fit(designm, ratings)
    vpreds=regr.predict(validatedm)
    mae=mean_absolute_error(validaterats, vpreds)
    vdict[a]=mae
    rdict[a]=regr
    mybar.update()

minerroralpha=min(vdict, key=vdict.get)
print minerroralpha
regr=rdict[minerroralpha] 

regr.intercept_, ybar

testdm, _ = getmats(test_df)

predictions['baseline_r']=regr.predict(testdm)

np.savetxt("baseline_r.csv", predictions['baseline_r'], delimiter=",")"""